In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.8 MB/s eta 0:00

In [2]:
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string
nltk.download('stopwords')
from tqdm.auto import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Pre-processing and baseline

In [11]:
df=pd.read_csv("my_amazon_data")

In [12]:
df.head()

,Unnamed: 0,label,title,content
0,0,1,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,1,1,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,1,Amazing!,This soundtrack is my favorite music of all ti...
3,3,1,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,4,1,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."


In [13]:
df['Combined'] = df.apply(lambda row: row['title'] + ' ' + row['content'], axis=1)
df_train=df.iloc[0:int(df.shape[0]*0.7)]
df_test=df.iloc[int(df.shape[0]*0.7):]

In [8]:
# from sklearn.model_selection import train_test_split
# X_train_original, X_valid_original, y_train_original, y_valid_original = train_test_split(df_train['text'], df_train['label'], test_size=0.3, random_state=0)

In [14]:
X_train_original=df_train['Combined']
X_valid_original=df_test['Combined']
y_train_original=df_train['label']
y_valid_original=df_test['label']

In [15]:
import torch
assert torch.cuda.is_available()
device = torch.device("cuda")

In [16]:
from transformers import AutoTokenizer, AdamW,BertForSequenceClassification

In [17]:
tokenizer= AutoTokenizer.from_pretrained("bert-base-uncased")

In [18]:
X_train=tokenizer(list(X_train_original),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_original)

In [19]:
X_valid=tokenizer(list(X_valid_original),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_valid=list(y_valid_original)

In [20]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, token):
        self.token = token
    def __getitem__(self, id):
        return {key: val[id] for key, val in self.token.items()}
    def __len__(self):
        return len(self.token.input_ids)

In [21]:
bat_size=128

In [22]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
valid_dataloader = DataLoader(MyData(X_valid) , batch_size=bat_size,shuffle = False)

In [23]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [24]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids,
                                token_type_ids,
                                attention_mask,
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|          | 0/50 [00:00<?, ?it/s]

trainset:   0%|          | 0/6 [00:00<?, ?it/s]

validset:   0%|          | 0/3 [00:00<?, ?it/s]

epoch: 0
train loss: 0.926221509774526
valid loss: 0.8460355997085571
valid accuracy: 0.6133333333333333
——————————————————————————————————————————————


trainset:   0%|          | 0/6 [00:00<?, ?it/s]

...

validset:   0%|          | 0/3 [00:00<?, ?it/s]

epoch: 29
train loss: 0.15645190762976804
valid loss: 0.7055787245432535
valid accuracy: 0.7466666666666667
——————————————————————————————————————————————


trainset:   0%|          | 0/6 [00:00<?, ?it/s]

validset:   0%|          | 0/3 [00:00<?, ?it/s]

early stop


In [26]:
print(max(valid_accuracy_list))

0.8066666666666666


## Prompt 1(Baseline): Please rephrase the following sentence +text

In [27]:
!pip install openai
!pip install auto_tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for auto_tqdm: filename=auto_tqdm-1.0.3-py3-none-any.whl size=3265 sha256=9903d42b39d463167bcc1370160f8382e747d0c740060ca98e7b549e29bf9677
  Stored in directory: /root/.cache/pip/wheels/f2/65/e9/e282bd548e8f4f98b6faa115f1f871ded115b5561aac8b5801
  Created wheel for environments_utils: filename=environments_utils-1.0.10-py3-none-any.whl size=7299 sha256=5eb15b5306349756d749fdce465c408b5d4340f0e6b0edc1909847c7d30e517c
  Stored in directory: /root/.cache/pip/wheels/b6/83/4a/6b22ed373d85d32a7b1ee1c412f11dafa397da3d54e8ec8549
  Created wheel for support_developer: filename=support_

In [28]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentence: " +text}])
    return completion.choices[0].message.content

In [ ]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgpt(X_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1
        except:
            time.sleep(0.5)

In [ ]:
base_dict={"text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_amazon_baseline")

In [30]:
new_df=pd.read_csv("chatgpt_amazon_baseline")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [31]:
model_base = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model_base.to(device)
optimizer = AdamW(model_base.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [32]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model_base.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model_base.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model_base(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model_base.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model_base(input_ids,
                                token_type_ids,
                                attention_mask,
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model_base.state_dict(), "my_model_base")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model_base.state_dict(), "my_model_base")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


trainset: 100%|██████████| 11/11 [00:12<00:00,  1.14s/it]
                                                         
epoch:   2%|▏         | 1/50 [00:14<11:43, 14.36s/it]

epoch: 0
train loss: 1.0054809559475293
valid loss: 0.8253172834714254
valid accuracy: 0.49333333333333335
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:12<00:00,  1.16s/it]
                                                         
epoch:   4%|▍         | 2/50 [00:28<11:35, 14.48s/it]

epoch: 1
train loss: 0.7652492360635237
valid loss: 0.7219582001368204
valid accuracy: 0.6133333333333333
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:12<00:00,  1.17s/it]
                                                         
epoch:   6%|▌         | 3/50 [00:42<11:11, 14.30s/it]

...

epoch: 34
train loss: 0.22530752284960312
valid loss: 0.5914542277654012
valid accuracy: 0.7766666666666666
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
epoch:  72%|███████▏  | 36/50 [09:05<03:34, 15.34s/it]

epoch: 35
train loss: 0.1778459372845563
valid loss: 0.6034136315186819
valid accuracy: 0.76
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
validset: 100%|██████████| 3/3 [00:01<00:00,  2.89it/s]
                                                      

early stop


In [33]:
print(max(valid_accuracy_list))

0.8133333333333334


In [34]:
prompt1=max(valid_accuracy_list)

## Prompt 4: Please rephrase the following sentences from +source+ but keep their emotion +text

In [35]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt2(text, label):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    if label == 1:
      l="positive"
    elif label == 0:
      l="negative"
    else:
      l="neutral"
    content="Please rephrase this sentence while retaining the "+l+" emotion: "
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': content+text}])
    return completion.choices[0].message.content

In [36]:
rephrase_by_chatgpt2(X_train_original[1], y_train_original[1])

'I am absolutely amazed by this soundtrack! While many reviews label it as the best \'game soundtrack", I respectfully disagree. In my opinion, it is Yasunori Mitsuda\'s ultimate masterpiece and the music is truly timeless. Even after several years of listening, its beauty remains intact. Though it is quite costly, I must say it is worth every penny. If you\'re going to splurge on a CD, this is the only one that I would highly recommend.'

In [37]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgpt2(X_train_original[i], y_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1
        except:
            time.sleep(0.5)

In [ ]:
base_dict={"text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_amazon_label")

In [38]:
new_df=pd.read_csv("chatgpt_amazon_label")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [39]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [40]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids,
                                token_type_ids,
                                attention_mask,
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


trainset: 100%|██████████| 11/11 [00:12<00:00,  1.14s/it]
                                                         
epoch:   2%|▏         | 1/50 [00:14<11:34, 14.18s/it]

epoch: 0
train loss: 0.8931211070580916
valid loss: 0.7091153462727865
valid accuracy: 0.6433333333333333
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:12<00:00,  1.16s/it]
                                                         
epoch:   4%|▍         | 2/50 [00:28<11:26, 14.30s/it]

epoch: 1
train loss: 0.7245039018717679
valid loss: 0.6943358381589254
valid accuracy: 0.69
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:12<00:00,  1.17s/it]
                                                         
epoch:   6%|▌         | 3/50 [00:42<11:06, 14.18s/it]

...

epoch: 11
train loss: 0.18625448982823978
valid loss: 0.5487997134526571
valid accuracy: 0.79
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.28s/it]
                                                         
epoch:  26%|██▌       | 13/50 [03:13<09:16, 15.03s/it]

epoch: 12
train loss: 0.1754126481034539
valid loss: 0.6132027705510458
valid accuracy: 0.7666666666666667
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
validset: 100%|██████████| 3/3 [00:01<00:00,  2.92it/s]
                                                      

early stop


In [41]:
print(max(valid_accuracy_list))

0.8033333333333333


In [42]:
prompt4=max(valid_accuracy_list)

## Prompt 2: Please rephrase the following sentences from +source+ but keep their emotion +text

In [43]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt3(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentence from Amazon but keep their emotion: " +text}])
    return completion.choices[0].message.content

In [44]:
rephrase_by_chatgpt3(X_train_original[0])

"Wow, this soundtrack is absolutely amazing! The way it evokes scenes in your mind is incredible and even those who dislike video game music will surely appreciate it. I've played a lot of games, but Chrono Cross has the best music by far. Rather than relying on simple keyboard sounds, it incorporates fantastic guitar and orchestral elements that are simply beautiful. Anyone who takes the time to listen to it will be thoroughly impressed. ^_^"

In [45]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgpt3(X_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1
        except:
            time.sleep(0.5)

In [ ]:
base_dict={"text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_amazon_3")

In [46]:

new_df=pd.read_csv("chatgpt_amazon_3")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [47]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [48]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids,
                                token_type_ids,
                                attention_mask,
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


trainset: 100%|██████████| 11/11 [00:12<00:00,  1.14s/it]
                                                         
epoch:   2%|▏         | 1/50 [00:14<11:35, 14.20s/it]

epoch: 0
train loss: 0.9744629697366194
valid loss: 0.7535928090413412
valid accuracy: 0.6466666666666666
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:12<00:00,  1.16s/it]
                                                         
epoch:   4%|▍         | 2/50 [00:28<11:11, 13.99s/it]

epoch: 1
train loss: 0.7603975967927412
valid loss: 0.7400330702463785
valid accuracy: 0.4866666666666667
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:12<00:00,  1.17s/it]
                                                         
epoch:   6%|▌         | 3/50 [00:41<10:57, 13.98s/it]

...

epoch: 15
train loss: 0.2591992372816259
valid loss: 0.585041364034017
valid accuracy: 0.7333333333333333
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
epoch:  34%|███▍      | 17/50 [04:13<08:18, 15.11s/it]

epoch: 16
train loss: 0.2551258450204676
valid loss: 0.5908478200435638
valid accuracy: 0.7333333333333333
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
validset: 100%|██████████| 3/3 [00:01<00:00,  2.91it/s]
                                                      

early stop


In [49]:
print(max(valid_accuracy_list))

0.7933333333333333


In [50]:
prompt2=max(valid_accuracy_list)

## Prompt3: Please rephrase the following sentences to around +max length words without changing emotion +text

In [51]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt4(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentences to around 64 words without changing emotion: "+ text}])
    return completion.choices[0].message.content

In [52]:
rephrase_by_chatgpt4(X_train_original[1])

'Amidst the rave reviews dubbing it the best "game soundtrack," I stand in discord- to me, Yasunori Mitsuda\'s creation is beyond a mere soundtrack. It\'s a masterpiece, its beauty eternal as I\'ve been listening to it for years and it never fails to astound. Yes, the price is steep, but if there\'s one CD deserving of the sum, it\'s this one. For me, it\'s simply unrivaled.'

In [53]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgpt4(X_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1
        except:
            time.sleep(0.5)

In [ ]:
base_dict={"text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_amazon_maxword")

In [54]:
new_df=pd.read_csv("chatgpt_amazon_maxword")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [55]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [56]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids,
                                token_type_ids,
                                attention_mask,
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


trainset: 100%|██████████| 11/11 [00:12<00:00,  1.14s/it]
                                                         
epoch:   2%|▏         | 1/50 [00:14<11:42, 14.35s/it]

epoch: 0
train loss: 1.0030964016914368
valid loss: 0.7888699571291605
valid accuracy: 0.6333333333333333
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:12<00:00,  1.16s/it]
                                                         
epoch:   4%|▍         | 2/50 [00:28<11:29, 14.37s/it]

epoch: 1
train loss: 0.7996407693082636
valid loss: 0.7355835835138956
valid accuracy: 0.65
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:12<00:00,  1.18s/it]
                                                         
epoch:   6%|▌         | 3/50 [00:42<11:09, 14.24s/it]

...

epoch: 11
train loss: 0.3757312460379167
valid loss: 0.6520799199740092
valid accuracy: 0.7
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.28s/it]
                                                         
epoch:  26%|██▌       | 13/50 [03:13<09:14, 15.00s/it]

epoch: 12
train loss: 0.39553374593908136
valid loss: 0.7957104643185934
valid accuracy: 0.62
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
validset: 100%|██████████| 3/3 [00:01<00:00,  2.91it/s]
                                                      

early stop


In [57]:
print(max(valid_accuracy_list))

0.7733333333333333


In [58]:
prompt3=max(valid_accuracy_list)

## Prompt 5: Please translate to Spainish and translate back

In [59]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt6(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please translate this sentence from English to Spainish while retaining the emotion: " +text}])
    return completion.choices[0].message.content

In [60]:
rephrase_by_chatgpt6(X_train_original[1])

'La mejor banda sonora que he escuchado en mi vida. He leído muchas reseñas diciendo que esta es la mejor "banda sonora de un videojuego", pero he decidido escribir para discrepar un poco. En mi opinión, este es el máximo trabajo maestro de Yasunori Mitsuda. La música es intemporal y la he estado escuchando durante años y su belleza simplemente se niega a desvanecerse. El precio de esto es bastante impresionante, debo decirlo, pero si vas a comprar algún cd por tanto dinero, este es el único que siento que valdría cada centavo.'

In [61]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgpt6(X_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1
        except:
            time.sleep(0.5)

In [62]:
base_dict={"text":x_new,"label":y_new}
new_df_spain=pd.DataFrame.from_dict(base_dict)
new_df_spain.to_csv("chatgpt_amazon_translate_spainish")
new_df_spain.head()

,text,label
0,¡Impresionante incluso para los no jugadores! ...,1
1,La mejor banda sonora de cualquier cosa. He es...,1
2,¡Increíble! Esta banda sonora es mi música fav...,1
3,Excelente banda sonora. Realmente me gusta est...,1
4,"Recuerda, ¡levanta tu mandíbula del suelo desp...",1


In [63]:
def rephrase_back(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please translate this sentence from Spainish to English: " +text}])
    return completion.choices[0].message.content

In [64]:
rephrase_back(new_df_spain['text'][1])

'The best soundtrack of anything. I have been reading many reviews saying that this is the best "game soundtrack" and I thought I would write a review to disagree a bit. In my opinion, this is Yasunori Mitsuda\'s ultimate masterpiece. The music is timeless and I have been listening to it for years and its beauty just refuses to fade. The price of this is quite amazing, I must say, but if you are going to buy any CD for this amount of money, this is the only one I feel is worth every penny.'

In [65]:
x_new=[]
y_new=[]
for i in tqdm(range(len(new_df_spain))):
    count=0
    # print(new_df_spain['text'][i])
    # print(new_df_spain['label'][i])
    while(count!=1):
        try:
            ans=rephrase_back(new_df_spain['text'][i])
            x_new.append(ans)
            y_new.append(new_df_spain['label'][i])
            count=1
        except:
            time.sleep(0.5)

In [66]:
base_dict={"text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.head()
new_df.to_csv("chatgpt_amazon_translate_to_english")

In [68]:
new_df=pd.read_csv("chatgpt_amazon_translate_to_english")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [ ]:
%%time
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad()
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids,
                            token_type_ids,
                            attention_mask,
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids,
                                token_type_ids,
                                attention_mask,
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

epoch: 0
train loss: 0.8684621507471258
valid loss: 0.7789791425069174
valid accuracy: 0.6466666666666666
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:13<00:00,  1.19s/it]
                                                         
epoch:   4%|▍         | 2/50 [00:29<11:53, 14.87s/it]

epoch: 1
train loss: 0.7585695656863126
valid loss: 0.7296368082364401
valid accuracy: 0.65
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:13<00:00,  1.22s/it]
                                                         
epoch:   6%|▌         | 3/50 [00:44<11:44, 14.99s/it]

...


trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
epoch:  52%|█████▏    | 26/50 [06:44<06:13, 15.56s/it]

epoch: 25
train loss: 0.2719131897796284
valid loss: 0.5426908532778422
valid accuracy: 0.7766666666666666
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
epoch:  54%|█████▍    | 27/50 [06:59<05:57, 15.54s/it]

epoch: 26
train loss: 0.22446287084709515
valid loss: 0.5716926554838816
valid accuracy: 0.7633333333333333
——————————————————————————————————————————————



trainset: 100%|██████████| 11/11 [00:14<00:00,  1.29s/it]
                                                         
validset:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(max(valid_accuracy_list))

In [ ]:
prompt5=max(valid_accuracy_list)